Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Categorization and Complex System Instructions

This notebook provides a simple example for interacting with Google's Gemini models for chat using Vertex AI. For more information please visit https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini

Import python libraries

In [ ]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part, Tool, ChatSession, FunctionDeclaration
import vertexai.preview.generative_models as generative_models
from vertexai.preview.generative_models import grounding
import requests
import json

Set your project variables. Change "YOUR_PROJECT_ID" to your GCP project ID.

In [ ]:
project_id = "YOUR_PROJECT_ID"
location = "global"
region = "us-central1"

# Define the prompt to request a recipe for a particular cuisine 

Change the folloiwng cuisine to test the categorization agent and generate a recipe. 

In [ ]:
prompt = f'''
Show me a recipe for making Jambalaya.
'''

# Define the Categorization Agent (i.e. cat_agent)

This will use an agent to define the categorization of the provided cuisine and return the response in a JSON object. The JSON object will include both a food_category value and a food_code value. These values will be used to provide a custom context to the recipe agent.

Define the response schema. In this example we will generate values for "category" and "code" to provide speficic system instructions to the Recipe agent based on the type of cuisine. Category will be an easily readable food category and used to provide additional information to the persona for the recipe model. Code will be used to provide specific information in the context section for the recipe model. 

In [ ]:
response_schema = {
    "type": "object",
    "properties": {
        "category": {
            "type": "string",
        },
        "code": {
            "type": "string",
        },
    },
    "required": ["category", "code"],
}

In [ ]:
cat_agent_config = GenerationConfig(
    #response_mime_type = "text/plain",
    response_mime_type = "application/json",
    temperature=0,
    #top_p=0.2,
    #top_k=2,
    #candidate_count=1,
    #max_output_tokens=8192,
    response_schema=response_schema
)

In [ ]:
cat_agent_sys_instruct = f'''
<PERSONA>
You are a professional chef, skilled in identifying different culinary regions and cuisines
</PERSONA>

<CONTEXT>
There are 7 categories and their associated code to choose from:
Louisiana Creole: louisiana_creole
Latin American: lat_am
Caribbean: carib
Indian: indian
Western European: west_euro
Fusion: fusion
Other: other
</CONTEXT>

<OBJECTIVE>
The goal is to correctly identify the culinary style and assign it to one of the 7 listed categories.
</OBJECTIVE>
'''


In [ ]:
cat_agent_model = GenerativeModel(
    "gemini-1.5-flash-002",
    generation_config=cat_agent_config,
    #tools=[tool_final],
    system_instruction=[cat_agent_sys_instruct],
)

# Generate the categorization

Generate the json data for category and code

In [ ]:
cat_agent = cat_agent_model.generate_content(prompt)

In [ ]:
print(cat_agent.text)

Load the json data and define the food_category and food_code variables

In [ ]:
cat_agent_data = json.loads(cat_agent.text)
food_category = cat_agent_data['category']
food_code = cat_agent_data['code']

In [ ]:
print(food_category)
print(food_code)

# Define special instructions based on the identified food category

As defined in the Categorization Agent's system instructions, the available categories and their associated codes are:

Louisiana Creole: louisiana_creole

Latin American: lat_am

Caribbean: carib

Indian: indian

Western European: west_euro

Fusion: fusion

Other: other

## Create a dictionary of the specific contexts

This dictionary contains the system context that will be used based on the identified food code.

In [ ]:
contexts = {
    'louisiana_creole': f'''
<CONTEXT>
This cuisine is a vibrant and flavorful culinary tradition born in Louisiana, particularly in New Orleans. 
It's a melting pot of influences, reflecting the diverse cultures that shaped the region – French, Spanish, African, Native American, and even a touch of Italian and German. 
This cuisine often reflects a more urban and refined sensibility, with influences from the regions more affluent kitchens.

Key characteristics of Creole cuisine include:
The "Holy Trinity":  Onions, celery, and bell peppers form the aromatic base of countless Creole dishes, much like mirepoix in French cuisine.
Roux: A thickening agent made by cooking flour in fat, usually butter or oil. The roux can range in color from blonde (for lighter sauces and soups) to dark brown (for gumbo and stews), imparting a rich flavor and texture.
Seafood: Given Louisiana's proximity to the Gulf of Mexico, seafood plays a prominent role, including shrimp, crab, oysters, crawfish, and fish.
Tomatoes: Tomatoes are a key ingredient in many Creole sauces and stews, adding acidity and sweetness.
Rice:  Rice is a staple, often served alongside main dishes or incorporated into jambalaya and other specialties.

The spices that truly make Louisiana Creole cuisine special are a complex and layered blend:
Cayenne Pepper: This provides the signature heat in many dishes, adding a fiery kick without being overly aggressive.
Black Pepper: Adds a robust, pungent flavor and depth.
White Pepper: Used for its milder heat and subtle earthy notes.
Paprika: Contributes a sweet, smoky flavor and vibrant color, especially Spanish paprika.
Garlic Powder:  Enhances the savory notes and adds a hint of pungency.
Onion Powder: Adds a sweet and savory base flavor.
Dried Thyme: Provides an earthy, slightly minty flavor that complements the other spices.
Dried Oregano: Adds a peppery, slightly bitter note that balances the sweetness.
Bay Leaves:  Used to infuse a subtle, herbal aroma into stews and sauces.
File Powder: A unique spice made from ground sassafras leaves, used as a thickener and flavoring agent in gumbo, adding a distinctive, slightly root beer-like flavor.
</CONTEXT>
    ''',
    'lat_am': f'''
<CONTEXT>
Latian American food is incredibly diverse, encompassing a vast range of cuisines influenced by Indigenous traditions, European colonization (primarily Spanish and Portuguese), African cultures, and Asian immigration.
Common characteristics of Latin American cuisines include:
Fresh Produce: Abundant fruits, vegetables, and herbs are essential, reflecting the rich biodiversity of the region.  Think tomatoes, corn, avocados, beans, peppers, cilantro, and more.
Meats: Beef, pork, poultry, and seafood are all popular, often marinated or seasoned with vibrant spice blends.
Corn: A staple grain, used in everything from tortillas and tamales to arepas and empanadas.
Beans: Another staple, providing protein and flavor in dishes like black beans in Cuba, kidney beans in Mexico, and lentils in Peru.
Chilies:  From mild to fiery, chilies are used extensively for heat, flavor, and color.

Key spices that contribute to the diverse flavors of Latin American cuisine:
Cumin: An earthy, warm spice used across the region, especially in Mexican, Cuban, and Tex-Mex cooking.
Chili Powder: Blends vary greatly, but often include dried chilies, cumin, oregano, and garlic. Ancho, chipotle, and New Mexico chilies are common.
Oregano: Both Mexican and Mediterranean oregano are used, adding a pungent, slightly bitter note.
Coriander (Cilantro): Fresh cilantro is a ubiquitous herb, adding a bright, citrusy flavor to salsas, guacamole, and many other dishes.
Achiote (Annato):  A seed that provides a vibrant red-orange color and a slightly earthy, peppery flavor, especially in Caribbean and Central American cuisines.
Garlic: Essential for building flavor in sofritos, marinades, and sauces.
Onion:  Another aromatic base ingredient, used in various forms.
Paprika: Adds sweetness and color, particularly smoked paprika in some regions.
Cinnamon: Used in both savory and sweet dishes, particularly in Mexican moles and Caribbean stews.
Cloves:  Appear in both savory and sweet preparations, especially in mole sauces.
Allspice:  Common in Caribbean cuisine, lending a warm, complex flavor that hints at cinnamon, cloves, and nutmeg.
</CONTEXT>
''',
    'carib': f'''
<CONTEXT>
Caribbean cuisine is a vibrant tapestry of flavors, reflecting the diverse history and cultural influences of the islands. 
Indigenous traditions, European colonization (Spanish, British, French, Dutch), African heritage, and Indian and Chinese immigration have all contributed to the unique culinary landscape of the Caribbean.

Spices that define Caribbean cuisine:
Scotch Bonnet Peppers:  These fiery chilies are a hallmark of Caribbean cooking, adding intense heat and fruity flavor.
Allspice:  Used extensively, particularly in Jamaican jerk seasoning, providing a warm, complex flavor with hints of cinnamon, cloves, and nutmeg.
Nutmeg:  Grown throughout the islands, nutmeg adds a warm, sweet, and slightly nutty flavor to both savory and sweet dishes.
Ginger:  Used fresh, dried, or ground, ginger adds a pungent, spicy kick to marinades, sauces, and desserts.
Cinnamon:  Found in both savory and sweet preparations, cinnamon contributes warmth and sweetness.
Cloves:  Used in stews, marinades, and desserts, adding a warm, aromatic flavor.
Cumin:  Appears in some dishes, particularly those with Indian or Spanish influences.
Thyme:  Used fresh or dried, thyme adds an earthy, slightly minty note to stews and marinades.
Garlic:  Essential for building flavor in sofritos and sauces.
Onion:  Another aromatic base ingredient, used in various forms.
Paprika:  Adds color and a mild sweetness to some dishes.

Regional variations in spice usage:
Jamaica:  Known for its jerk seasoning, a fiery blend typically including scotch bonnet peppers, allspice, thyme, and ginger.
Trinidad and Tobago:  Influenced by Indian cuisine, using curry powder, turmeric, and cumin.
Cuba:  Sofrito, a base of sautéed onions, garlic, and bell peppers, is common, along with cumin and oregano.
Dominican Republic:  Employs a similar sofrito, often with annatto (achiote) for color and flavor.
Puerto Rico:  Uses sofrito as well, with adobo seasoning (a blend of garlic powder, oregano, and other spices).
</CONTEXT>
''',
    'indian': f'''
<CONTEXT>
Indian cuisine is renowned globally for its incredible diversity and complex use of spices.  
Spanning a vast subcontinent with varied regional traditions and religious influences, Indian food is a vibrant tapestry of flavors, aromas, and textures.

Key characteristics of Indian cuisine:
Spices: The defining feature, used in intricate combinations to create depth, complexity, and balance.
Fresh Produce:  Fruits, vegetables, and herbs are essential, including onions, garlic, ginger, chilies, tomatoes, and cilantro.
Legumes: Lentils, chickpeas, and other beans are staple sources of protein.
Grains: Rice, wheat, and millet are foundational, used in various forms like roti, naan, and dosa.
Dairy Products: Yogurt, ghee (clarified butter), and paneer (cheese) are commonly used.
Regional Variations: From the creamy curries of the north to the fiery seafood dishes of the south, Indian cuisine offers immense regional diversity.

Essential spices in Indian cuisine:
Turmeric:  A cornerstone spice, providing a vibrant yellow color and earthy, slightly bitter flavor. It also has anti-inflammatory properties.
Cumin: Used whole or ground, cumin adds a warm, earthy flavor and is a staple in many dishes.
Coriander:  Both the seeds and fresh leaves (cilantro) are used, offering citrusy and fresh notes.
Chili Powder:  From mild Kashmiri chili to fiery ghost peppers, chili powders add heat and depth of flavor.
Garam Masala:  A blend of ground spices, varying by region but typically includes cinnamon, cardamom, cloves, black pepper, and nutmeg.
Ginger:  Used fresh, dried, or ground, ginger adds a pungent, spicy kick.
Garlic:  Essential for building flavor in curries and other dishes.
Cardamom:  Green cardamom offers a fragrant, slightly sweet flavor, while black cardamom is more smoky.
Cloves:  Provide a warm, sweet, and intense flavor.
Cinnamon:  Used in both savory and sweet preparations, adding warmth and sweetness.
Black Pepper:  Adds a pungent, spicy kick.
Mustard Seeds:  Used whole in tempering (tadka), adding a pungent, nutty flavor.
Fenugreek Seeds:  Provide a slightly bitter, maple-syrupy flavor.
Saffron:  A prized spice, adding a vibrant yellow color and a distinct, floral aroma.

Regional spice variations:
North India:  Known for rich, creamy curries using garam masala, cardamom, and saffron.
South India:  Employs more chilies, mustard seeds, curry leaves, and coconut.
East India:  Features mustard oil, panch phoron (a five-spice blend), and fish.
West India:  Uses coconut, chilies, and tamarind for tangy flavors.
</CONTEXT>
''',
    'west_euro': f'''
<CONTEXT>
Western European cuisine encompasses a wide range of culinary traditions, influenced by geography, history, and cultural exchange. While diverse, some common themes and spice 
usages tie the region together.

Key characteristics of Western European cuisine:
Fresh, Seasonal Ingredients:  Emphasis on locally sourced produce, meats, and seafood.
Dairy Products:  Cheese, butter, and cream are widely used.
Bread:  Various types of bread are staples, from crusty baguettes to hearty rye loaves.
Wine:  Plays a significant role in both cooking and dining.
Regional Variations:  Distinct culinary traditions exist within each country and even specific regions.
Commonly used spices in Western European cuisine:
Black Pepper: A staple across the region, adding pungency and depth of flavor.
Parsley: Used fresh or dried, adding a bright, herbaceous note.
Thyme: A versatile herb, providing an earthy, slightly minty flavor.
Rosemary:  Aromatic and slightly piney, often used with roasted meats and vegetables.
Bay Leaves:  Used to infuse a subtle, herbal aroma into stews and sauces.
Garlic: Essential for building flavor in many dishes.
Onion: Another aromatic base ingredient, used in various forms.
Nutmeg: Adds warmth and a slightly sweet flavor to savory dishes and desserts.
Cloves:  Used sparingly in savory dishes and more prominently in desserts and beverages.
Mustard:  Used as a condiment and in sauces, adding a pungent kick.
Juniper Berries:  Used to flavor game meats and in gin production.

Regional spice variations:
France:  Emphasizes fresh herbs like thyme, rosemary, and parsley, along with bay leaves and black pepper. Regional variations include saffron in the south and anise in the east.
Italy:  Known for its simplicity, relying on fresh herbs like basil, oregano, and rosemary, along with garlic, black pepper, and chili flakes in some regions.
Spain:  Uses smoked paprika (pimentón), saffron, garlic, and bay leaves.
Portugal:  Employs piri-piri chilies, bay leaves, garlic, and paprika.
United Kingdom:  Traditionally uses fewer spices, relying on herbs like thyme, rosemary, and sage, along with mustard and black pepper. Indian spices have become more prevalent due to historical ties.
Germany:  Uses caraway seeds, juniper berries, mustard, and black pepper.
Belgium & Netherlands:  Similar to Germany, with the addition of nutmeg and cloves in some dishes.

While Western European cuisine may not be as spice-forward as some other global cuisines, the careful use of herbs, spices, and aromatics creates balanced and flavorful dishes that reflect the region's diverse culinary heritage. 
The focus is often on enhancing the natural flavors of high-quality ingredients rather than masking them with overpowering spices.
</CONTEXT>
''',
    'fusion': f'''
<CONTEXT>
Fusion cuisine, by its very nature, is about blending culinary traditions and techniques from different cultures to create something new and exciting. 
Spices play a critical role in fusion cooking, acting as bridges between diverse flavor profiles and allowing chefs to create innovative and harmonious dishes.

Key aspects of fusion cuisine and spice usage:
Cross-Cultural Inspiration: Fusion dishes draw inspiration from two or more distinct culinary traditions, often blending ingredients, techniques, and spices in unexpected ways.
Creative License: Chefs have significant freedom to experiment and develop unique flavor combinations, pushing boundaries and challenging traditional notions of cuisine.
Balance and Harmony: The goal is to create dishes where the different components complement each other, resulting in a cohesive and delicious whole. Spices are crucial for achieving this balance.
Global Pantry: Fusion cooking often utilizes a wide range of spices from around the world, creating a global pantry of flavors.

Examples of spice usage in fusion cuisine:
Asian-Latin Fusion:  Combining the heat of chilies with the umami of soy sauce and ginger, perhaps in a dish like Korean BBQ tacos or a Peruvian ceviche with aji amarillo and lemongrass.
Mediterranean-Indian Fusion:  Blending the warmth of cumin and coriander with the brightness of oregano and lemon, perhaps in a dish like tandoori-spiced lamb with a Greek yogurt sauce or a chickpea curry with saffron and feta.
French-Japanese Fusion:  Marrying the delicate flavors of miso and yuzu with the richness of butter and herbs, perhaps in a dish like miso-glazed sea bass with a beurre blanc sauce or a green tea crème brûlée.
Mexican-Italian Fusion:  Combining the smokiness of chipotle peppers with the sweetness of tomatoes and basil, perhaps in a dish like chipotle pesto pasta or a pizza with chorizo and Oaxaca cheese.

Spices often act as the unifying element in fusion dishes:
Bridging Flavors: They can create a connection between seemingly disparate ingredients, for example, using turmeric to link coconut milk and chickpeas in a Thai-inspired curry.
Adding Complexity: Layering spices from different traditions can create depth and nuance in a dish, for example, using star anise, cinnamon, and cloves in a Vietnamese-inspired pho broth.
Creating Unexpected Twists: Introducing a spice from one cuisine into a dish from another can create a surprising and delightful element, for example, using smoked paprika in a Japanese ramen broth.
</CONTEXT>
''',
    'other': f'''
<CONTEXT>
Provide a recipe for making the desired food item.
</CONTEXT>
'''
}

Based on the identified category, the context we will be using in the Recipe agent is:

In [ ]:
print(contexts[food_code])

# Instantiate the Recipe Agent

This agent will use the output of the categorization agent and use the specific context for the identified food category to generate a recipe.

Define the generation config parameters for the recipe agent

In [ ]:
recipe_agent_config = GenerationConfig(
    response_mime_type = "text/plain",
    #response_mime_type = "application/json",
    temperature=1,
    top_p=0.5,
    top_k=3,
    #candidate_count=1,
    #max_output_tokens=8192,
    #response_schema=response_schema
)

You can see here that we are using the food_category value to guide the persona of the recipe agent and the food_code to specify the context.
This will help reduce the complexity of the context window while still allowing for a wide range of very specific instructions for the model.

In [ ]:
recipe_agent_sys_instruct = f'''
<PERSONA>
You are a culinary expert, specalizing in {food_category} cuisine.
</PERSONA>

{contexts[food_code]}

<OBJECTIVE>
Based on the provided context about '{food_category}', create a delicious and authentic recipe.
</OBJECTIVE>
'''


Now we will instantiate the recipe agent and generate the requested recipe

In [ ]:
recipe_agent_model = GenerativeModel(
    "gemini-1.5-pro-002",
    generation_config=recipe_agent_config,
    #tools=[tool_final],
    system_instruction=[recipe_agent_sys_instruct],
)

recipe_agent = recipe_agent_model.start_chat()

## Start with a simple chat session

We will start the recipe agent chat session with the same prompt we used for the categorization agent.

In [ ]:
response = recipe_agent.send_message(prompt)
print(response.text)

## Now let's continue the chat session with the recipe agent

In [ ]:
prompt = "I need to feed 10 people. How should I change the measurements?"
response = recipe_agent.send_message(prompt)
print(response.text)